## **GF Project 5**

**Project Goal and Objectives**: Have the bot every 30 mins from 9am - 9pm EST give an update on the traffic flow and compare those values with MAs.

### **Bot**:
 - 1) Trigger the file request using a trigger: Trigger + Playwirght 
 - 2) Call the notebook (NEF_Project_5_local.ipynb)
 - 3) Output the print message from the notebook to the chat
 - 4) Keep in the directory the files shown in ORIGINAL_FILES list

### **NEF_Project_%_local.ipynb**:
- 1) Will start by impoting the file (Traffic View 3)
- 2) Calculate all the present stats 
- 3) Insert into the databse the values for the given day and hour 
- 4) Comapre result with MAs from same time of previous last week, if standard deviaiton >20% --> ALERT 

#### STEP 1 - Importing Traffic View for a given instance 

In [50]:
import os 
import pandas as pd

current_dir = os.getcwd()
files = os.listdir(current_dir)
for file in files:
    if "Project5" in file and file.endswith('.csv'):
        print(file)
        file_name = file
        
tview1 = pd.read_csv(file_name)
tview1.head()



Project5.csv


,Item,<track>Tracking 2</track>,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
0,DP1 RP V3,Coleman GoKart,3,3,100.00%,0,0.00%,2,2,100.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$0.60,0.00%,$-0.60,0.00%
1,DP1 RP V3,Garmin Instinct 3 Amoled Limited Edition Watch,4,4,100.00%,0,0.00%,4,4,100.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$0.80,0.00%,$-0.80,0.00%
2,DP1 RP V3,Garmin Instinct 3 Limited Edition Watch,8,8,100.00%,0,0.00%,7,7,100.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$1.60,0.00%,$-1.60,0.00%
3,DP1 RP V3,GearWrench 18 Drawer Rolling Tool Cabinet,6,6,100.00%,0,0.00%,4,4,100.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$1.20,0.00%,$-1.20,0.00%
4,DP1 RP V3,Harley Davidson Evo Helmet,18,16,88.89%,2,11.11%,17,15,88.24%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$5.50,14.51%,$32.40,85.49%


## First we will calulate teh value of the Prepaids

### **DP1 PP / (Dp1 pp + dp1)**

NOTE: This will be done on a per product basis

# 1)First we have to iterate throguh each product

In [51]:
# Filter only for products that have over 25 sales for the day 
filtered_df = tview1[(tview1["Successful Charges"] > 25) & (tview1["Item"] == "DP1 V3")]
valid_products = filtered_df["<track>Tracking 2</track>"].unique()
print(valid_products)

# Delete all rows that have RP in the Item column 
tview1 = tview1[~tview1["Item"].str.contains("RP")]


# keep tview1 for ony these products 
tview1 = tview1[tview1["<track>Tracking 2</track>"].isin(valid_products)]
tview1








['Miller Lite Tailgate Combo Pack' 'Tito Tailgate bundle']


,Item,<track>Tracking 2</track>,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
36,DP1 V3,Miller Lite Tailgate Combo Pack,200,140,70.00%,60,30.00%,115,55,47.83%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$77.36,10.00%,$657.37,85.00%
41,DP1 V3,Tito Tailgate bundle,127,99,77.95%,28,22.05%,64,36,56.25%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$43.32,12.00%,$317.60,88.00%
54,DP1 V3(PP),Miller Lite Tailgate Combo Pack,9,5,55.56%,4,44.44%,9,5,55.56%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$3.52,6.83%,$48.04,93.17%
58,DP1 V3(PP),Tito Tailgate bundle,2,1,50.00%,1,50.00%,2,1,50.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$1.04,8.07%,$11.85,91.93%
73,DP12 V3,Miller Lite Tailgate Combo Pack,48,38,79.17%,10,20.83%,40,30,75.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$50.86,5.14%,$839.15,84.86%
82,DP12 V3(PP),Miller Lite Tailgate Combo Pack,1,0,0.00%,1,100.00%,1,0,0.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$5.14,5.20%,$93.75,94.80%


In [52]:
unique_products = tview1["<track>Tracking 2</track>"].unique()
print(unique_products)

['Miller Lite Tailgate Combo Pack' 'Tito Tailgate bundle']


In [53]:
unique_charges = tview1["Item"].unique()
print(unique_charges)

['DP1 V3' 'DP1 V3(PP)' 'DP12 V3' 'DP12 V3(PP)']


In [54]:
# First step: Keep only rows that contain 'DP1'
tview1 = tview1[tview1["Item"].str.contains('DP1')]

# Second step: Remove any rows that contain 'DP12', 'DP13', etc
tview1 = tview1[~tview1["Item"].str.contains('DP1[2-9]')]
unique_charges = tview1["Item"].unique()
print(unique_charges)

['DP1 V3' 'DP1 V3(PP)']


In [55]:
import pytz
from datetime import datetime

ny_time = datetime.now(pytz.timezone('America/New_York')).strftime("%#I:%M%p").lower()
print(ny_time)

1:34pm


In [56]:
message = f"📺 TV message (NY {ny_time})\n\n"
for product in unique_products:
    # Get product data
    df = tview1[tview1["<track>Tracking 2</track>"] == product]
    df_sucar = df[df["Item"] == 'DP1 V3']
    total_sales = df_sucar["Successful Charges"].sum()
    
    # Calculate Prepaid
    denom = df["Successful Charges"].sum()
    df_num = df[df["Item"].str.contains("PP")]
    num = df_num["Successful Charges"].sum()
    prepaid_percentage = 0 if num == 0 or denom == 0 else num / denom
    prepaid_percentage = prepaid_percentage * 100
    prepaid_percentage = round(prepaid_percentage, 2)
    
    # Get SUCAR and convert to numeric value for comparison
    sucar = df_sucar["Success Unique Cycle Attempt Rate"].values[0]
    sucar_value = float(sucar.strip('%')) if isinstance(sucar, str) else float(sucar)
    
    # Format product section
    message += f"{product}:\n"
    message += f"∘ Initials: {total_sales}\n"
    
    # Only show SUCAR value if no alert
    if sucar_value < 50:
        message += f"⚠️ ALERT: SUCAR is below 50% ({sucar_value:.0f}%)\n"
    else:
        message += f"∘ SUCAR: {sucar}\n"
        
    if prepaid_percentage == 0:
        message += f"∘ Prepaid: 0.00% (no data available)\n"
    else:
        if prepaid_percentage > 10:
            message += f"⚠️ ALERT: Prepaids are above 10% ({prepaid_percentage:.2f}%)\n"
        else:
            message += f"∘ Prepaid: {prepaid_percentage:.2f}%\n"
    message += "\n"

    
output_message = message

print(output_message)

📺 TV message (NY 1:34pm)

Miller Lite Tailgate Combo Pack:
∘ Initials: 60
∘ SUCAR: 52.17%
∘ Prepaid: 6.25%

Tito Tailgate bundle:
∘ Initials: 28
⚠️ ALERT: SUCAR is below 50% (44%)
∘ Prepaid: 3.45%


